In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response'

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming 'data' is your loaded dataset

# Separating features and targets
features = data[['LADDLE_TEMP', 'CASTING_TEMP', ...]]  # List all feature columns
targets = data[['Carbon(C%)', 'Sulphur(S%)']]  # Target columns

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Initialize the multi-target regressor model (e.g., RandomForestRegressor as the base regressor)
base_regressor = RandomForestRegressor()
multi_output_regressor = MultiOutputRegressor(base_regressor)

# Train the multi-target regression model
multi_output_regressor.fit(X_train, y_train)

# Predict on the test set
predictions = multi_output_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")


NameError: name 'data' is not defined

In [5]:
# entity 

from dataclasses import dataclass 
from pathlib import Path 



@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path
    model_name: str
    imputer_name: str
    n_estimators: float
    max_depth: float
    target_column: str  

In [6]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [7]:
# configuration
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema = self.schema

        FileOperations.create_directories([config.root_dir])

        target_values = schema.get('TARGET_COLUMN', {})
        target_column = {
            'target_1': target_values.get('target_1'),
            'target_2': target_values.get('target_2')
        }

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            imputer_name = config.imputer_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            target_column=target_column  # Assign target_1 and target_2 to target_column
        )

        return model_trainer_config


In [8]:
# defining component

import pandas as pd
import joblib
import os 
from src.Mlflow_Project.__init__ import logger 
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error



In [11]:

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        target_column_1 = self.config.target_column['target_1']
        target_column_2 = self.config.target_column['target_2']

        X_train = train_data.drop([target_column_1, target_column_2], axis=1)
        X_test = test_data.drop([target_column_1, target_column_2], axis=1)
        y_train = train_data[[target_column_1, target_column_2]]
        y_test = test_data[[target_column_1, target_column_2]]

        y_imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
        y_train_imputed = y_imputer.fit_transform(y_train)
        y_test_imputed = y_imputer.transform(y_test)

        param_grid = {
            'estimator__n_estimators': [50, 100, 150],
            'estimator__max_depth': [None, 5, 10, 15],
            # Other parameters...
        }

        base_regressor = RandomForestRegressor(random_state=42)

        multi_output_regressor = MultiOutputRegressor(base_regressor)

        random_search = RandomizedSearchCV(
            estimator=multi_output_regressor,
            param_distributions=param_grid,
            n_iter=10,
            scoring='neg_mean_squared_error',
            cv=5,
            random_state=42
        )

        random_search.fit(X_train, y_train_imputed)

        best_estimator = random_search.best_estimator_

        predictions = best_estimator.predict(X_test)

        mse = mean_squared_error(y_test_imputed, predictions)
        print(f"Mean Squared Error: {mse}")

        test_x = pd.DataFrame(X_test, columns=X_test.columns)
        test_y = pd.DataFrame(y_test_imputed, columns=[target_column_1, target_column_2])

        # Save the scaled datasets and target variables
        test_x.to_csv(os.path.join(self.config.root_dir, "test_x.csv"), index=False)
        test_y.to_csv(os.path.join(self.config.root_dir, "test_y.csv"), index=False)

        joblib.dump(best_estimator, os.path.join(self.config.root_dir, self.config.model_name))
        joblib.dump(y_imputer, os.path.join(self.config.root_dir, self.config.imputer_name))


In [12]:
# Defining Pipeline 

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    print("An error occurred:", e)

[2023-11-17 09:21:13,122: INFO: yaml file: config\config.yaml loaded successfully]
[2023-11-17 09:21:13,126: INFO: yaml file: params.yaml loaded successfully]
[2023-11-17 09:21:13,132: INFO: yaml file: schema.yaml loaded successfully]
[2023-11-17 09:21:13,132: INFO: created directory at: artifacts]
[2023-11-17 09:21:13,132: INFO: created directory at: artifacts/model_trainer]
Mean Squared Error: 4.5245488051403895e-05
